In [ ]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
from load_functions import *

In [ ]:
test_users = pd.read_csv('test_users.csv', dtype=str, names=['user_id'])
users_df = load_users('users.csv', test_users)
orgs_df = load_orgs('organisations.csv')

In [ ]:
aspects = load_aspects()
rubrics = load_rubrics(orgs_df=orgs_df)
features = load_features(orgs_df=orgs_df)

In [ ]:
reviews, user_agg, org_agg = load_reviews(users_df=users_df, orgs_df=orgs_df)
orgs_df = orgs_df.merge(org_agg, on='org_id')
users_df = users_df.merge(user_agg, on='user_id')

In [ ]:
display(orgs_df.describe())
fig, axs = plt.subplots(nrows=6 ,figsize = (25,16))
sns.ecdfplot(data=orgs_df, x='average_bill',ax=axs[0])
sns.histplot(data=orgs_df, x='rating', ax=axs[1])
sns.histplot(data=orgs_df, x='rubrics_l', ax=axs[2], hue='city', stat='percent', bins=40)
sns.histplot(data=orgs_df, x='features_l', ax=axs[3], hue='city', stat='percent',bins=40)
sns.histplot(data=orgs_df, x='n_reviews', ax=axs[4], hue='city', stat='percent',bins=40)
sns.histplot(data=orgs_df, x='n_travels', ax=axs[5], hue='city', stat='percent',bins=40)

In [ ]:
g = sns.FacetGrid(orgs_df, col="city",height=10, aspect=.7)
g.map(sns.scatterplot, "average_bill","rating")

In [ ]:
fig, axs = plt.subplots(nrows=3 ,figsize = (15,16))
sns.ecdfplot(data=review_stats[review_stats.n_ratings<10], x='n_ratings', stat='proportion', ax=axs[0])
sns.histplot(data=review_stats, x='mean_rating', stat='proportion', ax=axs[1], bins=6)
sns.ecdfplot(data=review_stats[review_stats.mean_aspects<4], x='mean_aspects', stat='proportion', ax=axs[2])

In [ ]:
fig, axs = plt.subplots(nrows=2 ,figsize = (15,16))
sns.histplot(data=reviews, x='in_test', stat='percent', ax=axs[0])
sns.histplot(data=reviews, x='travel', stat='percent', ax=axs[1])

In [ ]:
sns.histplot(count_stat, x='rating', bins=[1,2,3,5,10,50], stat='percent')